In [45]:
import tensorflow as tf
!pip install transformers
from os import path, environ
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np
import os
import nltk
import glob
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords

[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

You should consider upgrading via the 'C:\Users\USER\Desktop\School Work\COM 726\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [46]:
# merging the files
joined_files = os.path.join(r"\Users\USER\OneDrive - Solent University\School Work\COM 726\AE1\Dataset\Monzo", "exported_mz*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
df_2 = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df_2

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,15cae506-4176-4364-b94d-d0053cc1345a,Martin,https://play-lh.googleusercontent.com/a/ALm5wu...,Sucks. Impossible to reset pin.,1,1,4.52.0,2022-11-30 14:34:38,Hey Martin 👋 Sorry to hear you're having troub...,2022-11-30 15:03:24
1,99996806-f16b-4085-bfce-8cf5f910e9bb,Kev Minds,https://play-lh.googleusercontent.com/a/ALm5wu...,Monzo app is always spot on and uploads really...,5,0,5.2.0,2022-11-30 13:00:20,NaN,NaN
2,bfa1d602-a3e8-441e-a222-9b5304912ad2,Samuel David,https://play-lh.googleusercontent.com/a-/ACNPE...,Unable to fil in my id for days after several ...,2,0,5.2.0,2022-11-30 10:52:40,"Hey Samuel 👋 We're so sorry to hear that, how ...",2022-11-30 10:54:32
3,6a0e0d02-4a4e-438f-896d-353343f6612e,Michael “Mitch” McBain,https://play-lh.googleusercontent.com/a-/ACNPE...,glitch-free,5,0,5.1.0,2022-11-29 23:47:01,NaN,NaN
4,09f8fb25-0640-49ad-b854-57f32fef06f4,Glenn Lewis,https://play-lh.googleusercontent.com/a-/ACNPE...,"Have to speak as I find, I have been using Mon...",5,0,4.45.1,2022-11-29 22:45:22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
342938,47640aee-6c66-4564-8841-2b59e1ef911c,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It's a waiting list. Wow! Absolutely pointless...,1,1,0.1.2,2016-05-25 11:45:04,NaN,NaN
342939,1d0e1714-7ec4-4e0f-aa68-34f4fa720fa7,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Pretty disappointed to go from an email saying...,2,3,0.1.2,2016-05-25 11:34:15,NaN,NaN
342940,6fb7a8d3-4ee4-4025-a69c-05a0d36a4e8b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,On the waiting list. Looking forward to all th...,5,8,0.1.2,2016-05-25 11:31:56,NaN,NaN
342941,e347d6bf-e867-489d-a9a6-f20a142e9368,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Mondo for Android - can't wait. The iOS app i...,5,4,1.0,2016-05-13 15:05:25,NaN,NaN


In [47]:
#printing shape of the entire dataframe
df_2.shape

(342943, 10)

In [48]:
#Creating a new column called sentiment by using the review rating score
def sentiment(n):return 1 if n >= 3 else 0
df_2['sentiment'] = df_2['score'].apply(sentiment)

In [50]:
df_2.tail(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sentiment
342938,47640aee-6c66-4564-8841-2b59e1ef911c,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It's a waiting list. Wow! Absolutely pointless...,1,1,0.1.2,2016-05-25 11:45:04,NaN,NaN,0
342939,1d0e1714-7ec4-4e0f-aa68-34f4fa720fa7,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Pretty disappointed to go from an email saying...,2,3,0.1.2,2016-05-25 11:34:15,NaN,NaN,0
342940,6fb7a8d3-4ee4-4025-a69c-05a0d36a4e8b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,On the waiting list. Looking forward to all th...,5,8,0.1.2,2016-05-25 11:31:56,NaN,NaN,1
342941,e347d6bf-e867-489d-a9a6-f20a142e9368,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Mondo for Android - can't wait. The iOS app i...,5,4,1.0,2016-05-13 15:05:25,NaN,NaN,1
342942,6d6d57b6-c109-49ff-ba28-4b0eb9a9785b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It is just the waiting list for now but it is ...,5,7,1.0,2016-05-13 11:30:09,NaN,NaN,1


In [55]:
#Viewing the  data frame columns of ony content and sentiment
df_new= df_2[['content','sentiment']]
df_new.tail(10)

,content,sentiment
342933,I was on iOS and had mondo - now on android an...,1
342934,All good knowing something is happening all my...,1
342935,No idea why as an investor we are being made t...,0
342936,"Great to see something is being done, nice ide...",1
342937,Brilliant way to minimise losses by gaging int...,1
342938,It's a waiting list. Wow! Absolutely pointless...,0
342939,Pretty disappointed to go from an email saying...,0
342940,On the waiting list. Looking forward to all th...,1
342941,Mondo for Android - can't wait. The iOS app i...,1
342942,It is just the waiting list for now but it is ...,1


In [56]:
#Check if there are any null values
#data_1 = data[['verified_reviews','sentiment']]
df_new.isnull().sum()

#There are 23 null values in the content column

content      23
sentiment     0
dtype: int64

In [57]:
# Drop duplicates
df_new.drop_duplicates("content", inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_11740\228607245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.drop_duplicates("content", inplace=True)


In [59]:
df_new.shape

(12885, 2)

In [60]:
#dropping empty  rows in the content column
df_new.dropna(inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_11740\2735915399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.dropna(inplace=True)


In [61]:
#Getting the shape of the data set after removing null values
df_new.shape

(12884, 2)

In [62]:
#Putting the content and sentiment into dataframe
reviews = df_new['content'].values.tolist()
labels = df_new['sentiment'].tolist()

In [63]:
#Printing some columns
print(reviews[:2])
print(labels[:2])

['Sucks. Impossible to reset pin.', 'Monzo app is always spot on and uploads really quickly']
[0, 1]


In [64]:
#Splitting into test and training
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [66]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [67]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=128)

{'input_ids': [[101, 5310, 5379, 10439, 2029, 7906, 2017, 7172, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [68]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [69]:

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [70]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [85]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2


AttributeError: in user code:

    File "C:\Users\USER\Desktop\School Work\COM 726\venv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\USER\Desktop\School Work\COM 726\venv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        
    File "C:\Users\USER\Desktop\School Work\COM 726\venv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        
    File "C:\Users\USER\Desktop\School Work\COM 726\venv\lib\site-packages\transformers\modeling_tf_utils.py", line 1476, in train_step
        data = data_adapter.expand_1d(data)

    AttributeError: module 'keras.engine.data_adapter' has no attribute 'expand_1d'


In [82]:
model.save_pretrained("./sentiment")

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/sentiment/")

In [ ]:

test_sentence = "This is a really bad product. I hate it"



predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = loaded_model.predict(predict_input)[0]


tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])